In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import math
from catboost import CatBoostRegressor
from sklearn import ensemble
from sklearn.preprocessing import RobustScaler, StandardScaler
from xgboost.sklearn import XGBClassifier, XGBRegressor
import pickle

## 模型训练 

In [2]:
import pandas as pd
np.random.seed(2022)

In [3]:
train = pd.read_csv('train.csv')
train = train[['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
              'V_10', 'treatment', 'outcome']]

feature_columns=['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
       'V_10', 'treatment']
# treatment_columns_continuous=['V_6', 'V_7']
treatment_columns_category=['V_10', 'treatment']
treatment_columns_common=['V_6','treatment']
treatment_change_value=0.2

outcome_column=["outcome"]
treatment_columns_continuous={
  "V_6": [None, 10, 1],
  "V_7": [0.3, None, 0]
}

In [4]:
# 众数 补充缺失值
for col in train.columns:
    d = dict(train[col].value_counts())
    train[col] = train[col].fillna(
        sorted(d.items(), key=lambda x: x[1], reverse=True)[0][0])

In [5]:
treatment_columns_category_dict=dict()
for col in treatment_columns_category:
    label_encoder = LabelEncoder().fit(train[col])
    temp_dict = dict(zip(range(0,len(label_encoder.classes_)),label_encoder.classes_)) 
    treatment_columns_category_dict[col]=temp_dict
    train[col] = label_encoder.transform(train[col])

In [6]:
treatment_columns_category_dict

{'V_10': {0: 'no', 1: 'unknown', 2: 'yes'}, 'treatment': {0: 0, 1: 1, 2: 2}}

In [7]:
treatment_columns_category_dict

{'V_10': {0: 'no', 1: 'unknown', 2: 'yes'}, 'treatment': {0: 0, 1: 1, 2: 2}}

In [8]:
for col in train.columns:
    if train[col].dtype == 'object':
        label_encoder = LabelEncoder().fit(train[col])
        train[col] = label_encoder.transform(train[col])

In [9]:
xgb = XGBRegressor()
xgb.fit(train[feature_columns],train[outcome_column])

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [10]:
train["outcome1"] = xgb.predict(train[feature_columns])

In [11]:
train

,V_0,V_1,V_2,V_3,V_4,V_5,V_6,V_7,V_8,V_9,V_10,treatment,outcome,outcome1
0,1.778505,2.376919,0.0,-1.680176,-1.581142,0.0,-31.4,0.838,0,3,0,2,4.745129,7.181605
1,-0.631728,5.337631,0.0,-0.869348,-2.767838,0.0,-42.0,4.120,0,999,0,0,7.310963,-9.275015
2,-0.862595,3.215953,1.0,-1.614799,-2.614270,1.0,-41.8,4.866,0,999,0,2,-52.156193,-29.456884
3,0.230854,2.250193,0.0,-1.631994,-5.705525,1.0,-36.1,4.966,0,999,2,0,-17.011566,-29.824350
4,0.194774,4.896442,0.0,-0.143322,2.568728,1.0,-42.7,4.968,0,999,2,2,-15.229788,-17.985956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36183,0.826893,4.331358,0.0,-1.551377,-2.904315,0.0,-46.2,1.250,0,999,1,2,20.512318,16.280304
36184,0.631707,1.383925,0.0,-0.411007,4.483282,1.0,-42.7,4.962,0,999,0,0,-3.178066,-15.284721
36185,-0.072426,4.918745,0.0,-0.351575,-4.926085,1.0,-36.1,4.963,0,999,0,0,-43.733190,-29.804337
36186,-1.029582,4.633457,0.0,-0.496135,3.944163,1.0,-46.2,1.334,0,999,0,2,12.905083,13.052897


In [12]:
# def convert(pred,t_ori,outcome,flag):
#     if (flag == 'treat1' and t_ori == '1') or (flag == 'treat2' and t_ori == '2') or (flag == 'treat0' and t_ori == '0'):
#         return outcome
#     else:
#         return pred
# train['t1_final'] = train.apply(lambda x:convert(x['t1_pred'],x['t_ori'],x['outcome'],'treat1'),axis=1) 
# train['t2_final'] = train.apply(lambda x:convert(x['t2_pred'],x['t_ori'],x['outcome'],'treat2'),axis=1) 
# train['t0_final'] = train.apply(lambda x:convert(x['t0_pred'],x['t_ori'],x['outcome'],'treat0'),axis=1) 

##    根据treatment_columns_category 和 treatment_columns_continuous 求uplift

In [13]:
# feature_columns=['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
#        'V_10', 'treatment']

# treatment_columns_category=['V_10', 'treatment']

# # treatment_columns_common=['V_6','treatment']    ###一定要变化的特征，不用参与比较

# treatment_columns_common=[]

# treatment_change_value=0.2

# outcome_column=["outcome"]

# treatment_columns_continuous={
#   "V_6": [None, 10, 1],
#   "V_7": [None, 3.9, 1]
# }  ##连续值的treatment， ["minValue", "maxValue", "negCorrelation_boolean"]

In [14]:
feature_columns

['V_0',
 'V_1',
 'V_2',
 'V_3',
 'V_4',
 'V_5',
 'V_6',
 'V_7',
 'V_8',
 'V_9',
 'V_10',
 'treatment']

In [15]:
outcome_pd=pd.DataFrame()
outcome_pd['outcome']=xgb.predict(train[feature_columns])
result=dict()

###  基于连续型变量的处理

In [16]:
treatment_cont_common = []
for col, limit_value in treatment_columns_continuous.items():
    if col in treatment_columns_common:
        treatment_cont_common.append(col)
    else:
        train_temp = train.copy(deep=True)
        if limit_value[2] == 0:
            if limit_value[0] != None:
                train_temp[col] = train_temp[col].apply(
                    lambda x: x*(1-treatment_change_value) if x*(1-treatment_change_value) > limit_value[0] else x)
            if limit_value[1] != None:
                train_temp[col] = train_temp[col].apply(
                    lambda x: x*(1-treatment_change_value) if x*(1-treatment_change_value) < limit_value[1] else x)
        if limit_value[2] == 1:
            if limit_value[0] != None:
                train_temp[col] = train_temp[col].apply(
                    lambda x: x*(1+treatment_change_value) if x*(1+treatment_change_value) > limit_value[0] else x)
            if limit_value[1] != None:
                train_temp[col] = train_temp[col].apply(
                    lambda x: x*(1+treatment_change_value) if x*(1+treatment_change_value) < limit_value[1] else x)
        outcome_pd[col] = xgb.predict(train_temp[feature_columns])
result["treatment_cont_common"] = treatment_cont_common

In [17]:
treatment_cont_common

['V_6']

In [21]:
result

{'treatment_cont_common': ['V_6']}

### 基于类别型变量的处理

In [20]:
treatment_cate_common = []
for col in treatment_columns_category:
    train_temp = train.copy(deep=True)
    if col in treatment_columns_common:
        pd_category = pd.DataFrame()
        print("{} is category 变量，在common中 ".format(col))
        treat_enum = treatment_columns_category_dict[col]
        print(treat_enum)
        for enum_value in treat_enum.keys():
            print(enum_value)
            train_temp[col] = enum_value
            pd_category[enum_value] = xgb.predict(train_temp[feature_columns])
        d = dict(pd_category.mean())
        d = sorted(d.items(), key=lambda x: x[1], reverse=True)[0][0]
        _temp_dict=dict()
        _temp_dict["treatment"]=col
        _temp_dict["value"]=treat_enum[d]
        treatment_cate_common.append(_temp_dict)
        print("\n\n")

    else:
        pd_category = pd.DataFrame()
        print("{} is category 变量，不在common中 ".format(col))
        treat_enum = treatment_columns_category_dict[col]
        print(treat_enum)
        for enum_value in treat_enum.keys():
            print(enum_value)
            train_temp[col] = enum_value
            pd_category[enum_value] = xgb.predict(train_temp[feature_columns])
        print("\n\n")
        cate_max = pd_category.max(axis=1)
        cate_idxmax = pd_category.idxmax(axis=1)
        pd_category[col] = cate_max
        pd_category[col+"_idx"] = cate_idxmax

        outcome_pd[col] = cate_max
        outcome_pd[col+"_idx"] = cate_idxmax

result["treatment_cate_common"] = treatment_cate_common

V_10 is category 变量，不在common中 
{0: 'no', 1: 'unknown', 2: 'yes'}
0
1
2



treatment is category 变量，在common中 
{0: 0, 1: 1, 2: 2}
0
1
2





In [23]:
_temp_dict

{'treatment': 'treatment', 'value': 2}

In [24]:
treatment_cate_common

[{'treatment': 'treatment', 'value': 2}]

In [25]:
temp_pd=outcome_pd[[i for i in treatment_columns_category+list(treatment_columns_continuous.keys()) if i not in treatment_columns_common]]
outcome_pd['max_idx'] = temp_pd.idxmax(axis=1)

In [26]:
train["user_id"]=train.index

In [27]:
outcome_pd["user_id"]=train["user_id"]

In [28]:
outcome_pd

,outcome,V_7,V_10,V_10_idx,max_idx,user_id
0,7.181605,7.661075,12.066837,1,V_10,0
1,-9.275015,-21.025122,12.434505,1,V_10,1
2,-29.456884,-15.164906,-12.047970,1,V_10,2
3,-29.824350,-33.163967,0.096378,1,V_10,3
4,-17.985956,-10.735950,5.339548,1,V_10,4
...,...,...,...,...,...,...
36183,16.280304,16.919127,16.280304,1,V_7,36183
36184,-15.284721,-18.526529,1.381678,1,V_10,36184
36185,-29.804337,-23.170704,-9.212712,1,V_10,36185
36186,13.052897,4.231896,34.504608,1,V_10,36186


In [29]:
outcome_pd["user_id"]=outcome_pd.index

In [36]:
user_res = []
outcome_pd_cols = outcome_pd.columns.to_list()
for key, value in outcome_pd.iterrows():
    _temp_dict=dict()
    max_idx = value["max_idx"]
    if max_idx+"_idx" in outcome_pd_cols:
        _temp_dict["user_id"]=value["user_id"]
        _temp_dict["treatment"]=max_idx
        _temp_dict["value"]=value[max_idx+"_idx"]
        
    else:
        _temp_dict["user_id"]=value["user_id"]
        _temp_dict["treatment"]=max_idx
    user_res.append(_temp_dict)
result["user_treatment"] = user_res

In [37]:
result

{'treatment_cont_common': ['V_6'],
 'treatment_cate_common': [{'treatment': 'treatment', 'value': 2}],
 'user_treatment': [{'user_id': 0, 'treatment': 'V_10', 'value': 1},
  {'user_id': 1, 'treatment': 'V_10', 'value': 1},
  {'user_id': 2, 'treatment': 'V_10', 'value': 1},
  {'user_id': 3, 'treatment': 'V_10', 'value': 1},
  {'user_id': 4, 'treatment': 'V_10', 'value': 1},
  {'user_id': 5, 'treatment': 'V_10', 'value': 1},
  {'user_id': 6, 'treatment': 'V_10', 'value': 1},
  {'user_id': 7, 'treatment': 'V_10', 'value': 1},
  {'user_id': 8, 'treatment': 'V_10', 'value': 1},
  {'user_id': 9, 'treatment': 'V_10', 'value': 1},
  {'user_id': 10, 'treatment': 'V_10', 'value': 1},
  {'user_id': 11, 'treatment': 'V_10', 'value': 1},
  {'user_id': 12, 'treatment': 'V_10', 'value': 1},
  {'user_id': 13, 'treatment': 'V_10', 'value': 1},
  {'user_id': 14, 'treatment': 'V_10', 'value': 1},
  {'user_id': 15, 'treatment': 'V_10', 'value': 1},
  {'user_id': 16, 'treatment': 'V_10', 'value': 1},
  {'u

In [35]:
# user_res = dict()
# outcome_pd_cols = outcome_pd.columns.to_list()
# for key, value in outcome_pd.iterrows():
#     max_idx = value["max_idx"]
#     if max_idx+"_idx" in outcome_pd_cols:
#         temp = dict()
#         temp[max_idx] = value[max_idx+"_idx"]
#         user_res[value["user_id"]] = temp
#     else:
#         user_res[value["user_id"]] = max_idx
# #     print(max_idx)
# result["user_treatment"] = user_res

### 保存结果

In [26]:
# 保存文件
with open("causal_inference.pkl", "wb") as tf:
    pickle.dump(result,tf)
    
    
# 读取文件
with open("causal_inference.pkl", "rb") as tf:
    new_dict = pickle.load(tf)

# print(new_dict)

In [44]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [46]:
# 保存文件
with open("causal_inference.json","w",encoding="utf8") as tf:
    json.dump(result,tf,ensure_ascii=False, indent=2, cls=NpEncoder)

In [3]:
train_data_path = args.
inference_data_path = args.
feature_columns = eval(args.)
treatment_columns_category = eval(args.)
treatment_columns_continuous = eval(args.)
treatment_columns_common = eval(args.)
outcome_column = eval(args.)
userid_column = eval(args.)


dict

In [8]:
with open('parameters.json','r',encoding='utf8')as fp:
    parameters = json.load(fp)

train_data_path=parameters["train_data_path"]
inference_data_path=parameters["inference_data_path"]
feature_columns=parameters["feature_columns"]
treatment_columns_category=parameters["treatment_columns_category"]
treatment_columns_continuous=parameters["treatment_columns_continuous"]
treatment_columns_common=parameters["treatment_columns_common"]
outcome_column=parameters["outcome_column"]
userid_column=parameters["userid_column"]

In [9]:
inference_data_path

'inference_data.csv'

In [18]:
outcome_column

['outcome']